In [1]:
import csv
import numpy
import os
import yaml
import time

In [2]:
from nupic.algorithms.sdr_classifier_factory import SDRClassifierFactory
from nupic.algorithms.spatial_pooler import SpatialPooler
from nupic.algorithms.temporal_memory import TemporalMemory
from nupic.encoders.random_distributed_scalar import RandomDistributedScalarEncoder

In [3]:
_NUM_RECORDS = 7000

_WORK_DIR = os.getcwd()
_INPUT_FILE_PATH = os.path.join(_WORK_DIR, "dataset", "eyeState14400.csv")
_PARAMS_PATH = os.path.join(_WORK_DIR, "params", "modelEyeState128.yaml")

In [4]:
with open(_PARAMS_PATH, "r") as f:
    modelParams = yaml.safe_load(f)["modelParams"]
    enParams = modelParams["sensorParams"]["encoders"]
    spParams = modelParams["spParams"]
    tmParams = modelParams["tmParams"]

In [5]:
encoderAF3 = RandomDistributedScalarEncoder(enParams["sensAF3"]["resolution"])
encoderF7 = RandomDistributedScalarEncoder(enParams["sensF7"]["resolution"])
encoderF3 = RandomDistributedScalarEncoder(enParams["sensF3"]["resolution"])
encoderFC5 = RandomDistributedScalarEncoder(enParams["sensFC5"]["resolution"])
encoderT7 = RandomDistributedScalarEncoder(enParams["sensT7"]["resolution"])
encoderP7 = RandomDistributedScalarEncoder(enParams["sensP7"]["resolution"])
encoderO1 = RandomDistributedScalarEncoder(enParams["sensO1"]["resolution"])
encoderO2 = RandomDistributedScalarEncoder(enParams["sensO2"]["resolution"])
encoderP8 = RandomDistributedScalarEncoder(enParams["sensP8"]["resolution"])
encoderT8 = RandomDistributedScalarEncoder(enParams["sensT8"]["resolution"])
encoderFC6 = RandomDistributedScalarEncoder(enParams["sensFC6"]["resolution"])
encoderF4 = RandomDistributedScalarEncoder(enParams["sensF4"]["resolution"])
encoderF8 = RandomDistributedScalarEncoder(enParams["sensF8"]["resolution"])
encoderAF4 = RandomDistributedScalarEncoder(enParams["sensAF4"]["resolution"])
encoderEyeDetection = RandomDistributedScalarEncoder(enParams["sensEyeDetection"]["resolution"])

encodingWidth = (encoderAF3.getWidth() + encoderF7.getWidth() + encoderF3.getWidth()
                 + encoderFC5.getWidth() + encoderT7.getWidth() + encoderP7.getWidth()
                 + encoderO1.getWidth() + encoderO2.getWidth() + encoderP8.getWidth()
                 + encoderT8.getWidth() + encoderFC6.getWidth() + encoderF4.getWidth()
                 + encoderF8.getWidth() + encoderAF4.getWidth() + encoderEyeDetection.getWidth())

In [6]:
start = time.time()

sp = SpatialPooler(
  inputDimensions=(encodingWidth,),
  columnDimensions=(spParams["columnCount"],),
  potentialPct=spParams["potentialPct"],
  potentialRadius=encodingWidth,
  globalInhibition=spParams["globalInhibition"],
  localAreaDensity=spParams["localAreaDensity"],
  numActiveColumnsPerInhArea=spParams["numActiveColumnsPerInhArea"],
  synPermInactiveDec=spParams["synPermInactiveDec"],
  synPermActiveInc=spParams["synPermActiveInc"],
  synPermConnected=spParams["synPermConnected"],
  boostStrength=spParams["boostStrength"],
  seed=spParams["seed"],
  wrapAround=False
)

end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("Waktu eksekusi : {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

Waktu eksekusi : 00:00:03.71


In [7]:
tm = TemporalMemory(
  columnDimensions=(tmParams["columnCount"],),
  cellsPerColumn=tmParams["cellsPerColumn"],
  activationThreshold=tmParams["activationThreshold"],
  initialPermanence=tmParams["initialPerm"],
  connectedPermanence=spParams["synPermConnected"],
  minThreshold=tmParams["minThreshold"],
  maxNewSynapseCount=tmParams["newSynapseCount"],
  permanenceIncrement=tmParams["permanenceInc"],
  permanenceDecrement=tmParams["permanenceDec"],
  predictedSegmentDecrement=0.0,
  maxSegmentsPerCell=tmParams["maxSegmentsPerCell"],
  maxSynapsesPerSegment=tmParams["maxSynapsesPerSegment"],
  seed=tmParams["seed"]
)

In [8]:
classifier = SDRClassifierFactory.create()
postvDetect = 0

In [9]:
start = time.time()

with open(_INPUT_FILE_PATH, "r") as fin:
    reader = csv.reader(fin)
    headers = reader.next()
    reader.next()
    reader.next()

    for count, record in enumerate(reader):

        if count >= _NUM_RECORDS:
            break

        sensAF3 = float(record[0])
        sensF7 = float(record[1])
        sensF3 = float(record[2])
        sensFC5 = float(record[3])
        sensT7 = float(record[4])
        sensP7 = float(record[5])
        sensO1 = float(record[6])
        sensO2 = float(record[7])
        sensP8 = float(record[8])
        sensT8 = float(record[9])
        sensFC6 = float(record[10])
        sensF4 = float(record[11])
        sensF8 = float(record[12])
        sensAF4 = float(record[13])
        sensEyeDetection = float(record[14])

        sensAF3_Bits = numpy.zeros(encoderAF3.getWidth())
        sensF7_Bits = numpy.zeros(encoderF7.getWidth())
        sensF3_Bits = numpy.zeros(encoderF3.getWidth())
        sensFC5_Bits = numpy.zeros(encoderFC5.getWidth())
        sensT7_Bits = numpy.zeros(encoderT7.getWidth())
        sensP7_Bits = numpy.zeros(encoderP7.getWidth())
        sensO1_Bits = numpy.zeros(encoderO1.getWidth())
        sensO2_Bits = numpy.zeros(encoderO2.getWidth())
        sensP8_Bits = numpy.zeros(encoderP8.getWidth())
        sensT8_Bits = numpy.zeros(encoderT8.getWidth())
        sensFC6_Bits = numpy.zeros(encoderFC6.getWidth())
        sensF4_Bits = numpy.zeros(encoderF4.getWidth())
        sensF8_Bits = numpy.zeros(encoderF8.getWidth())
        sensAF4_Bits = numpy.zeros(encoderAF4.getWidth())
        sensEyeDetection_Bits = numpy.zeros(encoderEyeDetection.getWidth())

        encoderAF3.encodeIntoArray(sensAF3, sensAF3_Bits)
        encoderF7.encodeIntoArray(sensF7, sensF7_Bits)
        encoderF3.encodeIntoArray(sensF3, sensF3_Bits)
        encoderFC5.encodeIntoArray(sensFC5, sensFC5_Bits)
        encoderT7.encodeIntoArray(sensT7, sensT7_Bits)
        encoderP7.encodeIntoArray(sensP7, sensP7_Bits)
        encoderO1.encodeIntoArray(sensO1, sensO1_Bits)
        encoderO2.encodeIntoArray(sensO2, sensO2_Bits)
        encoderP8.encodeIntoArray(sensP8, sensP8_Bits)
        encoderT8.encodeIntoArray(sensT8, sensT8_Bits)
        encoderFC6.encodeIntoArray(sensFC6, sensFC6_Bits)
        encoderF4.encodeIntoArray(sensF4, sensF4_Bits)
        encoderF8.encodeIntoArray(sensF8, sensF8_Bits)
        encoderAF4.encodeIntoArray(sensAF4, sensAF4_Bits)
        encoderEyeDetection.encodeIntoArray(sensEyeDetection,
                                            sensEyeDetection_Bits)

        encoding = numpy.concatenate(
            [sensAF3_Bits, sensF7_Bits, sensF3_Bits, sensFC5_Bits, sensT7_Bits,
             sensP7_Bits, sensO1_Bits, sensO2_Bits, sensP8_Bits, sensT8_Bits,
             sensFC6_Bits, sensF4_Bits, sensF8_Bits, sensAF4_Bits,
             sensEyeDetection_Bits]
        )

        activeColumns = numpy.zeros(spParams["columnCount"])

        sp.compute(encoding, True, activeColumns)
        activeColumnIndices = numpy.nonzero(activeColumns)[0]

        tm.compute(activeColumnIndices, learn=True)

        activeCells = tm.getActiveCells()

        bucketIdx = encoderEyeDetection.getBucketIndices(sensEyeDetection)[0]

        classifierResult = classifier.compute(
            recordNum=count,
            patternNZ=activeCells,
            classification={
                "bucketIdx": bucketIdx,
                "actValue": sensEyeDetection
            },
            learn=True,
            infer=True
        )

        oneStepConfidence, oneStep = sorted(
            zip(classifierResult[1], classifierResult["actualValues"]),
            reverse=True
        )[0]

        if (sensEyeDetection == oneStep):
            postvDetect += 1

        print("{:5}\tAktual: {:2.0f}\tHTM: {:2.0f}\t\tAkurasi: {:4.4}%".format(count, sensEyeDetection,
              oneStep, float(postvDetect) / (count+1) * 100))
        
        
end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("Waktu eksekusi : {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

    0	Aktual:  0	HTM:  0		Akurasi: 100.0%
    1	Aktual:  0	HTM:  0		Akurasi: 100.0%
    2	Aktual:  0	HTM:  0		Akurasi: 100.0%
    3	Aktual:  0	HTM:  0		Akurasi: 100.0%
    4	Aktual:  0	HTM:  0		Akurasi: 100.0%
    5	Aktual:  0	HTM:  0		Akurasi: 100.0%
    6	Aktual:  0	HTM:  0		Akurasi: 100.0%
    7	Aktual:  0	HTM:  0		Akurasi: 100.0%
    8	Aktual:  0	HTM:  0		Akurasi: 100.0%
    9	Aktual:  0	HTM:  0		Akurasi: 100.0%
   10	Aktual:  0	HTM:  0		Akurasi: 100.0%
   11	Aktual:  0	HTM:  0		Akurasi: 100.0%
   12	Aktual:  0	HTM:  0		Akurasi: 100.0%
   13	Aktual:  0	HTM:  0		Akurasi: 100.0%
   14	Aktual:  0	HTM:  0		Akurasi: 100.0%
   15	Aktual:  0	HTM:  0		Akurasi: 100.0%
   16	Aktual:  0	HTM:  0		Akurasi: 100.0%
   17	Aktual:  0	HTM:  0		Akurasi: 100.0%
   18	Aktual:  0	HTM:  0		Akurasi: 100.0%
   19	Aktual:  0	HTM:  0		Akurasi: 100.0%
   20	Aktual:  0	HTM:  0		Akurasi: 100.0%
   21	Aktual:  0	HTM:  0		Akurasi: 100.0%
   22	Aktual:  0	HTM:  0		Akurasi: 100.0%
   23	Aktual:  0	HTM:  0		Akurasi:

  199	Aktual:  1	HTM:  1		Akurasi: 93.5%
  200	Aktual:  1	HTM:  1		Akurasi: 93.53%
  201	Aktual:  1	HTM:  1		Akurasi: 93.56%
  202	Aktual:  1	HTM:  1		Akurasi: 93.6%
  203	Aktual:  1	HTM:  1		Akurasi: 93.63%
  204	Aktual:  1	HTM:  1		Akurasi: 93.66%
  205	Aktual:  1	HTM:  1		Akurasi: 93.69%
  206	Aktual:  1	HTM:  1		Akurasi: 93.72%
  207	Aktual:  1	HTM:  1		Akurasi: 93.75%
  208	Aktual:  1	HTM:  1		Akurasi: 93.78%
  209	Aktual:  1	HTM:  1		Akurasi: 93.81%
  210	Aktual:  1	HTM:  1		Akurasi: 93.84%
  211	Aktual:  1	HTM:  1		Akurasi: 93.87%
  212	Aktual:  1	HTM:  1		Akurasi: 93.9%
  213	Aktual:  1	HTM:  1		Akurasi: 93.93%
  214	Aktual:  1	HTM:  1		Akurasi: 93.95%
  215	Aktual:  1	HTM:  1		Akurasi: 93.98%
  216	Aktual:  1	HTM:  1		Akurasi: 94.01%
  217	Aktual:  1	HTM:  1		Akurasi: 94.04%
  218	Aktual:  1	HTM:  1		Akurasi: 94.06%
  219	Aktual:  1	HTM:  1		Akurasi: 94.09%
  220	Aktual:  1	HTM:  1		Akurasi: 94.12%
  221	Aktual:  1	HTM:  1		Akurasi: 94.14%
  222	Aktual:  1	HTM:  0		Akurasi: 93

  410	Aktual:  1	HTM:  0		Akurasi: 65.45%
  411	Aktual:  1	HTM:  0		Akurasi: 65.29%
  412	Aktual:  1	HTM:  0		Akurasi: 65.13%
  413	Aktual:  1	HTM:  0		Akurasi: 64.98%
  414	Aktual:  1	HTM:  0		Akurasi: 64.82%
  415	Aktual:  1	HTM:  0		Akurasi: 64.66%
  416	Aktual:  1	HTM:  1		Akurasi: 64.75%
  417	Aktual:  1	HTM:  1		Akurasi: 64.83%
  418	Aktual:  1	HTM:  1		Akurasi: 64.92%
  419	Aktual:  1	HTM:  1		Akurasi: 65.0%
  420	Aktual:  1	HTM:  0		Akurasi: 64.85%
  421	Aktual:  1	HTM:  0		Akurasi: 64.69%
  422	Aktual:  1	HTM:  0		Akurasi: 64.54%
  423	Aktual:  1	HTM:  0		Akurasi: 64.39%
  424	Aktual:  1	HTM:  0		Akurasi: 64.24%
  425	Aktual:  1	HTM:  0		Akurasi: 64.08%
  426	Aktual:  1	HTM:  0		Akurasi: 63.93%
  427	Aktual:  1	HTM:  0		Akurasi: 63.79%
  428	Aktual:  1	HTM:  1		Akurasi: 63.87%
  429	Aktual:  1	HTM:  0		Akurasi: 63.72%
  430	Aktual:  1	HTM:  0		Akurasi: 63.57%
  431	Aktual:  1	HTM:  0		Akurasi: 63.43%
  432	Aktual:  1	HTM:  0		Akurasi: 63.28%
  433	Aktual:  1	HTM:  0		Akurasi: 

  624	Aktual:  1	HTM:  1		Akurasi: 70.4%
  625	Aktual:  1	HTM:  1		Akurasi: 70.45%
  626	Aktual:  1	HTM:  1		Akurasi: 70.49%
  627	Aktual:  1	HTM:  1		Akurasi: 70.54%
  628	Aktual:  1	HTM:  1		Akurasi: 70.59%
  629	Aktual:  1	HTM:  1		Akurasi: 70.63%
  630	Aktual:  1	HTM:  1		Akurasi: 70.68%
  631	Aktual:  1	HTM:  1		Akurasi: 70.73%
  632	Aktual:  1	HTM:  1		Akurasi: 70.77%
  633	Aktual:  1	HTM:  1		Akurasi: 70.82%
  634	Aktual:  1	HTM:  1		Akurasi: 70.87%
  635	Aktual:  1	HTM:  1		Akurasi: 70.91%
  636	Aktual:  1	HTM:  1		Akurasi: 70.96%
  637	Aktual:  1	HTM:  1		Akurasi: 71.0%
  638	Aktual:  1	HTM:  1		Akurasi: 71.05%
  639	Aktual:  1	HTM:  1		Akurasi: 71.09%
  640	Aktual:  1	HTM:  1		Akurasi: 71.14%
  641	Aktual:  1	HTM:  1		Akurasi: 71.18%
  642	Aktual:  1	HTM:  1		Akurasi: 71.23%
  643	Aktual:  1	HTM:  1		Akurasi: 71.27%
  644	Aktual:  1	HTM:  1		Akurasi: 71.32%
  645	Aktual:  1	HTM:  1		Akurasi: 71.36%
  646	Aktual:  1	HTM:  1		Akurasi: 71.41%
  647	Aktual:  1	HTM:  1		Akurasi: 7

  839	Aktual:  0	HTM:  1		Akurasi: 77.62%
  840	Aktual:  0	HTM:  1		Akurasi: 77.53%
  841	Aktual:  0	HTM:  1		Akurasi: 77.43%
  842	Aktual:  0	HTM:  1		Akurasi: 77.34%
  843	Aktual:  0	HTM:  1		Akurasi: 77.25%
  844	Aktual:  0	HTM:  1		Akurasi: 77.16%
  845	Aktual:  0	HTM:  1		Akurasi: 77.07%
  846	Aktual:  0	HTM:  1		Akurasi: 76.98%
  847	Aktual:  0	HTM:  1		Akurasi: 76.89%
  848	Aktual:  0	HTM:  1		Akurasi: 76.8%
  849	Aktual:  0	HTM:  1		Akurasi: 76.71%
  850	Aktual:  0	HTM:  1		Akurasi: 76.62%
  851	Aktual:  0	HTM:  1		Akurasi: 76.53%
  852	Aktual:  0	HTM:  1		Akurasi: 76.44%
  853	Aktual:  0	HTM:  1		Akurasi: 76.35%
  854	Aktual:  0	HTM:  1		Akurasi: 76.26%
  855	Aktual:  0	HTM:  1		Akurasi: 76.17%
  856	Aktual:  0	HTM:  1		Akurasi: 76.08%
  857	Aktual:  0	HTM:  1		Akurasi: 75.99%
  858	Aktual:  0	HTM:  1		Akurasi: 75.9%
  859	Aktual:  0	HTM:  1		Akurasi: 75.81%
  860	Aktual:  0	HTM:  1		Akurasi: 75.73%
  861	Aktual:  0	HTM:  1		Akurasi: 75.64%
  862	Aktual:  0	HTM:  1		Akurasi: 7

...

 6039	Aktual:  0	HTM:  0		Akurasi: 66.85%
 6040	Aktual:  0	HTM:  1		Akurasi: 66.84%
 6041	Aktual:  0	HTM:  0		Akurasi: 66.85%
 6042	Aktual:  0	HTM:  0		Akurasi: 66.85%
 6043	Aktual:  0	HTM:  0		Akurasi: 66.86%
 6044	Aktual:  0	HTM:  0		Akurasi: 66.87%
 6045	Aktual:  0	HTM:  0		Akurasi: 66.87%
 6046	Aktual:  0	HTM:  0		Akurasi: 66.88%
 6047	Aktual:  0	HTM:  0		Akurasi: 66.88%
 6048	Aktual:  0	HTM:  1		Akurasi: 66.87%
 6049	Aktual:  0	HTM:  1		Akurasi: 66.86%
 6050	Aktual:  0	HTM:  1		Akurasi: 66.85%
 6051	Aktual:  0	HTM:  1		Akurasi: 66.84%
 6052	Aktual:  0	HTM:  1		Akurasi: 66.83%
 6053	Aktual:  0	HTM:  1		Akurasi: 66.82%
 6054	Aktual:  0	HTM:  1		Akurasi: 66.8%
 6055	Aktual:  0	HTM:  1		Akurasi: 66.79%
 6056	Aktual:  0	HTM:  1		Akurasi: 66.78%
 6057	Aktual:  0	HTM:  1		Akurasi: 66.77%
 6058	Aktual:  0	HTM:  1		Akurasi: 66.76%
 6059	Aktual:  0	HTM:  1		Akurasi: 66.75%
 6060	Aktual:  0	HTM:  0		Akurasi: 66.75%
 6061	Aktual:  0	HTM:  0		Akurasi: 66.76%
 6062	Aktual:  0	HTM:  0		Akurasi: 

 6239	Aktual:  0	HTM:  0		Akurasi: 67.29%
 6240	Aktual:  0	HTM:  0		Akurasi: 67.3%
 6241	Aktual:  0	HTM:  0		Akurasi: 67.3%
 6242	Aktual:  0	HTM:  0		Akurasi: 67.31%
 6243	Aktual:  0	HTM:  0		Akurasi: 67.31%
 6244	Aktual:  0	HTM:  0		Akurasi: 67.32%
 6245	Aktual:  0	HTM:  0		Akurasi: 67.32%
 6246	Aktual:  0	HTM:  0		Akurasi: 67.33%
 6247	Aktual:  0	HTM:  0		Akurasi: 67.33%
 6248	Aktual:  0	HTM:  0		Akurasi: 67.34%
 6249	Aktual:  0	HTM:  0		Akurasi: 67.34%
 6250	Aktual:  0	HTM:  0		Akurasi: 67.35%
 6251	Aktual:  0	HTM:  0		Akurasi: 67.35%
 6252	Aktual:  0	HTM:  0		Akurasi: 67.36%
 6253	Aktual:  0	HTM:  0		Akurasi: 67.36%
 6254	Aktual:  0	HTM:  1		Akurasi: 67.35%
 6255	Aktual:  0	HTM:  0		Akurasi: 67.36%
 6256	Aktual:  0	HTM:  0		Akurasi: 67.36%
 6257	Aktual:  0	HTM:  0		Akurasi: 67.37%
 6258	Aktual:  0	HTM:  0		Akurasi: 67.37%
 6259	Aktual:  0	HTM:  0		Akurasi: 67.38%
 6260	Aktual:  0	HTM:  0		Akurasi: 67.39%
 6261	Aktual:  0	HTM:  0		Akurasi: 67.39%
 6262	Aktual:  0	HTM:  0		Akurasi: 6

 6448	Aktual:  1	HTM:  1		Akurasi: 68.18%
 6449	Aktual:  1	HTM:  1		Akurasi: 68.19%
 6450	Aktual:  1	HTM:  1		Akurasi: 68.19%
 6451	Aktual:  1	HTM:  1		Akurasi: 68.2%
 6452	Aktual:  1	HTM:  0		Akurasi: 68.19%
 6453	Aktual:  1	HTM:  1		Akurasi: 68.19%
 6454	Aktual:  1	HTM:  1		Akurasi: 68.2%
 6455	Aktual:  1	HTM:  1		Akurasi: 68.2%
 6456	Aktual:  1	HTM:  0		Akurasi: 68.19%
 6457	Aktual:  1	HTM:  0		Akurasi: 68.18%
 6458	Aktual:  1	HTM:  1		Akurasi: 68.18%
 6459	Aktual:  1	HTM:  1		Akurasi: 68.19%
 6460	Aktual:  1	HTM:  0		Akurasi: 68.18%
 6461	Aktual:  1	HTM:  0		Akurasi: 68.17%
 6462	Aktual:  1	HTM:  0		Akurasi: 68.16%
 6463	Aktual:  1	HTM:  0		Akurasi: 68.15%
 6464	Aktual:  1	HTM:  0		Akurasi: 68.14%
 6465	Aktual:  1	HTM:  0		Akurasi: 68.13%
 6466	Aktual:  1	HTM:  0		Akurasi: 68.12%
 6467	Aktual:  1	HTM:  0		Akurasi: 68.1%
 6468	Aktual:  1	HTM:  1		Akurasi: 68.11%
 6469	Aktual:  1	HTM:  0		Akurasi: 68.1%
 6470	Aktual:  1	HTM:  0		Akurasi: 68.09%
 6471	Aktual:  1	HTM:  1		Akurasi: 68.0

 6645	Aktual:  1	HTM:  1		Akurasi: 67.03%
 6646	Aktual:  1	HTM:  1		Akurasi: 67.04%
 6647	Aktual:  1	HTM:  1		Akurasi: 67.04%
 6648	Aktual:  1	HTM:  1		Akurasi: 67.05%
 6649	Aktual:  1	HTM:  1		Akurasi: 67.05%
 6650	Aktual:  1	HTM:  1		Akurasi: 67.06%
 6651	Aktual:  1	HTM:  0		Akurasi: 67.05%
 6652	Aktual:  1	HTM:  0		Akurasi: 67.04%
 6653	Aktual:  1	HTM:  1		Akurasi: 67.04%
 6654	Aktual:  1	HTM:  1		Akurasi: 67.05%
 6655	Aktual:  1	HTM:  1		Akurasi: 67.05%
 6656	Aktual:  1	HTM:  1		Akurasi: 67.06%
 6657	Aktual:  1	HTM:  1		Akurasi: 67.06%
 6658	Aktual:  1	HTM:  1		Akurasi: 67.07%
 6659	Aktual:  1	HTM:  0		Akurasi: 67.06%
 6660	Aktual:  1	HTM:  1		Akurasi: 67.06%
 6661	Aktual:  1	HTM:  1		Akurasi: 67.07%
 6662	Aktual:  1	HTM:  1		Akurasi: 67.07%
 6663	Aktual:  1	HTM:  0		Akurasi: 67.06%
 6664	Aktual:  1	HTM:  0		Akurasi: 67.05%
 6665	Aktual:  1	HTM:  1		Akurasi: 67.06%
 6666	Aktual:  1	HTM:  1		Akurasi: 67.06%
 6667	Aktual:  1	HTM:  1		Akurasi: 67.07%
 6668	Aktual:  1	HTM:  1		Akurasi:

 6852	Aktual:  1	HTM:  1		Akurasi: 67.27%
 6853	Aktual:  1	HTM:  1		Akurasi: 67.27%
 6854	Aktual:  1	HTM:  1		Akurasi: 67.28%
 6855	Aktual:  1	HTM:  1		Akurasi: 67.28%
 6856	Aktual:  1	HTM:  1		Akurasi: 67.29%
 6857	Aktual:  1	HTM:  1		Akurasi: 67.29%
 6858	Aktual:  1	HTM:  1		Akurasi: 67.3%
 6859	Aktual:  1	HTM:  1		Akurasi: 67.3%
 6860	Aktual:  1	HTM:  1		Akurasi: 67.31%
 6861	Aktual:  1	HTM:  1		Akurasi: 67.31%
 6862	Aktual:  1	HTM:  1		Akurasi: 67.32%
 6863	Aktual:  1	HTM:  1		Akurasi: 67.32%
 6864	Aktual:  1	HTM:  1		Akurasi: 67.33%
 6865	Aktual:  1	HTM:  1		Akurasi: 67.33%
 6866	Aktual:  1	HTM:  1		Akurasi: 67.34%
 6867	Aktual:  1	HTM:  1		Akurasi: 67.34%
 6868	Aktual:  1	HTM:  1		Akurasi: 67.35%
 6869	Aktual:  1	HTM:  1		Akurasi: 67.35%
 6870	Aktual:  1	HTM:  1		Akurasi: 67.36%
 6871	Aktual:  1	HTM:  1		Akurasi: 67.36%
 6872	Aktual:  1	HTM:  1		Akurasi: 67.37%
 6873	Aktual:  1	HTM:  1		Akurasi: 67.37%
 6874	Aktual:  1	HTM:  1		Akurasi: 67.37%
 6875	Aktual:  1	HTM:  1		Akurasi: 6